In [3]:
import numpy as np

# (1) 그리드월드 환경 설정 (4x4)
grid_size = 4
gamma = 1.0  # 감가율

# (2) 상태 가치 초기화
value_table = np.zeros((grid_size, grid_size))

# (3) 종료 상태(목적지)
terminal_states = [(0, 0), (grid_size - 1, grid_size - 1)]

# (4) 가능한 행동: (행,열)상, 하, 좌, 우
actions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# (5) 정책 초기화
policy = np.ones((grid_size, grid_size, len(actions))) / len(actions)
for i, j in terminal_states:
    policy[i, j] = np.zeros(len(actions))  # 종료 상태는 정책 없음

# (6) 정책 평가 함수 정의: 정해진 정책 하에서 각 상태의 기대 보상(value)을 벨만 방정식을 기반으로 반복 계산
def policy_evaluation(value_table, policy, iterations=100):
    for _ in range(iterations):
        new_value_table = np.copy(value_table) #상태 가치 값을 새롭게 갱신하기 위해 복사본 생성

        for i in range(grid_size):
            for j in range(grid_size):
                if (i, j) in terminal_states: #종료상태는 업데이트 하지 않음(가치고정)
                    continue
                value = 0

                for action_idx, action in enumerate(actions):        #현재 상태 (i, j)에서 각 행동(상, 하, 좌, 우)을 시도
                    next_i, next_j = i + action[0], j + action[1]

                    # 경계를 벗어나면 현재 위치 유지
                    if next_i < 0 or next_i >= grid_size or next_j < 0 or next_j >= grid_size:
                        next_i, next_j = i, j

                    reward = -1
                    prob = policy[i, j, action_idx]
                    value += prob * (reward + gamma * value_table[next_i, next_j]) #상태가치 벨만방정식

                new_value_table[i, j] = value

        value_table = new_value_table

    return value_table

# (7) 정책 개선 함수 정의:  주어진 상태 가치 함수 value_table을 기준으로 탐욕적(greedy)으로 최적 행동(액션)을 선택하여 정책 생성
def policy_improvement(value_table):
    new_policy = np.zeros((grid_size, grid_size, len(actions))) #정책 초기화

    for i in range(grid_size):
        for j in range(grid_size):
            if (i, j) in terminal_states:
                continue

            values = []
            for action in actions:
                next_i, next_j = i + action[0], j + action[1]

                if next_i < 0 or next_i >= grid_size or next_j < 0 or next_j >= grid_size:
                    next_i, next_j = i, j

                values.append(value_table[next_i, next_j]) #각 행동을 했을 때 도달하는 상태의 가치를 수집

            best_action_value = np.max(values)             #가장 높은 상태 가치로 이동하는 행동을 선택
            best_actions = [idx for idx, v in enumerate(values) if v == best_action_value] # 가장 높은 가치로 이동할 수 있는 행동 찾기
            
            # 탐욕적(Greedy) 행동 선택: 가장 좋은 행동에만 확률을 부여함 (탐욕적 선택) ex:policy[1, 2] = [0, 1, 0, 0] 
            for action_idx in best_actions:
                new_policy[i, j, action_idx] = 1 / len(best_actions) #여러 개일 경우 확률을 나눠서 부여함 (예: 2개면 각 0.5)

    return new_policy

# (8) 정책 반복(평가 및 개선 반복 수행)
def policy_iteration(iterations=10):
    global value_table, policy

    for _ in range(iterations):
        value_table = policy_evaluation(value_table, policy)
        policy = policy_improvement(value_table)

    return value_table, policy

# 실행 및 결과 출력
final_values, final_policy = policy_iteration()

# (9) 최종 상태 가치 테이블
print("최종 상태 가치 테이블:")
print(np.round(final_values, 2))

# (10) 최정 정책
print("\n최종 정책(각 상태에서의 행동 확률):")
for i in range(grid_size):
    for j in range(grid_size):
        print(f"상태 ({i},{j}): {final_policy[i,j]}")

최종 상태 가치 테이블:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

최종 정책(각 상태에서의 행동 확률):
상태 (0,0): [0. 0. 0. 0.]
상태 (0,1): [0. 0. 1. 0.]
상태 (0,2): [0. 0. 1. 0.]
상태 (0,3): [0.  0.5 0.5 0. ]
상태 (1,0): [1. 0. 0. 0.]
상태 (1,1): [0.5 0.  0.5 0. ]
상태 (1,2): [0.25 0.25 0.25 0.25]
상태 (1,3): [0. 1. 0. 0.]
상태 (2,0): [1. 0. 0. 0.]
상태 (2,1): [0.25 0.25 0.25 0.25]
상태 (2,2): [0.  0.5 0.  0.5]
상태 (2,3): [0. 1. 0. 0.]
상태 (3,0): [0.5 0.  0.  0.5]
상태 (3,1): [0. 0. 0. 1.]
상태 (3,2): [0. 0. 0. 1.]
상태 (3,3): [0. 0. 0. 0.]
